<a href="https://colab.research.google.com/github/rajeevo/ML_Keras_Intro_Keras_Tuner/blob/master/ML_Keras_Intro_Keras_Tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 1.) Import Libraries

In [2]:
import tensorflow as tf
from tensorflow import keras

import IPython

In [3]:
!pip install -q -U keras-tuner
import kerastuner as kt

     |████████████████████████████████| 61kB 1.8MB/s 


In [4]:
# 2.) Download and Prepare the Dataset

In [5]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data() 

4423680/4422102 [==============================] - 0s 0us/step


In [6]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

In [7]:
# 3.) Define the Model

In [12]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))
  
  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)
  model.add(keras.layers.Dense(units = hp_units, activation = 'relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer 
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 
  
  model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True), 
                metrics = ['accuracy'])
  
  return model

In [13]:
# 4.) Instantiate the Tuner and Perform Hypertuning

In [14]:
tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy', 
                     max_epochs = 10,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_to_kt')                       

In [15]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

In [16]:
tuner.search(img_train, label_train, epochs = 10, validation_data = (img_test, label_test), callbacks = [ClearTrainingOutput()])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}
""")

INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 192 and the optimal learning rate for the optimizer
is 0.001



In [17]:
# Build the model with the optimal hyperparameters and train it on the data
model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs = 10, validation_data = (img_test, label_test))

Epoch 1/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.4898 - accuracy: 0.8269 - val_loss: 0.4226 - val_accuracy: 0.8513
Epoch 2/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3693 - accuracy: 0.8656 - val_loss: 0.3758 - val_accuracy: 0.8657
Epoch 3/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3334 - accuracy: 0.8776 - val_loss: 0.3670 - val_accuracy: 0.8704
Epoch 4/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3071 - accuracy: 0.8860 - val_loss: 0.3654 - val_accuracy: 0.8724
Epoch 5/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2894 - accuracy: 0.8927 - val_loss: 0.3451 - val_accuracy: 0.8759
Epoch 6/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2742 - accuracy: 0.8984 - val_loss: 0.3351 - val_accuracy: 0.8815
Epoch 7/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2600 - accuracy: 0.9040 - val_loss: 0.3356 - val_accuracy: